<a href="https://colab.research.google.com/github/omermehboob/NIDS-using-machine-learning-and-deep-learning/blob/main/NIDS_(Artificial_Neural_Networks_%2B_AutoEncoders).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np

In [4]:
PATHS = [
    '/content/drive/MyDrive/CICIDS2017/CSVs/Monday-WorkingHours.pcap_ISCX.csv',
    '/content/drive/MyDrive/CICIDS2017/CSVs/Tuesday-WorkingHours.pcap_ISCX.csv',
    '/content/drive/MyDrive/CICIDS2017/CSVs/Wednesday-workingHours.pcap_ISCX.csv',
    '/content/drive/MyDrive/CICIDS2017/CSVs/Thursday-WorkingHours-Morning-WebAttacks.pcap_ISCX.csv',
    '/content/drive/MyDrive/CICIDS2017/CSVs/Thursday-WorkingHours-Afternoon-Infilteration.pcap_ISCX.csv',
    '/content/drive/MyDrive/CICIDS2017/CSVs/Friday-WorkingHours-Morning.pcap_ISCX.csv',
    '/content/drive/MyDrive/CICIDS2017/CSVs/Friday-WorkingHours-Afternoon-PortScan.pcap_ISCX.csv',
    '/content/drive/MyDrive/CICIDS2017/CSVs/Friday-WorkingHours-Afternoon-DDos.pcap_ISCX.csv'
    
]
df = pd.read_csv(PATHS[0])
for i in range(1,len(PATHS)):
    temp = pd.read_csv(PATHS[i],encoding="ISO-8859-1")
    df = pd.concat([df,temp])

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (0,1,3,6,84) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)
/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (85) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
m = df.loc[df[' Flow Packets/s'] != np.inf,' Flow Packets/s'].max()
df[' Flow Packets/s'].replace(np.inf,m,inplace=True)
m = df.loc[df['Flow Bytes/s'] != np.inf,'Flow Bytes/s'].max()
df['Flow Bytes/s'].replace(np.inf,m,inplace=True)

In [6]:
dtypes = df.dtypes
print(f"Number of columns with Int {len(dtypes[dtypes == int])}")
print(f"Number of columns with float {len(dtypes[dtypes == float])}")
print(f"Number of columns with object {len(dtypes[dtypes == object])}")

Number of columns with Int 0
Number of columns with float 80
Number of columns with object 6


In [7]:
null_values = df.isna().sum()
null_values[null_values >0]

Flow ID               288602
 Source IP            288602
 Source Port          288602
 Destination IP       288602
 Destination Port     288602
                      ...   
 Idle Std             288602
 Idle Max             288602
 Idle Min             288602
 Label                288602
External IP          3077509
Length: 86, dtype: int64

In [8]:
null_index = np.where(df['Flow Bytes/s'].isnull())[0]
df.dropna(inplace = True)

In [9]:
 labels = df[' Label'].unique()
 label_enc  = dict()
 for i in range(len(labels)):
     label_enc[labels[i]] = i
 df[' Label'] = df[' Label'].map(label_enc)
 import plotly.express as px
 fig = px.imshow(df.corr())
 fig.show()

In [11]:
df[' Destination Port']

18869     80.0
18870     80.0
18871     80.0
18872     80.0
18873     80.0
          ... 
196985    80.0
196986    80.0
196987    80.0
196988    80.0
196989    80.0
Name:  Destination Port, Length: 41836, dtype: float64

In [12]:
temp = df[df[' Label'] == 'BENIGN']
temp[' Destination Port'].describe()
temp = temp.sample(frac = 0.1)

In [13]:
df = df[df[' Label'] != 'BENIGN']
df = pd.concat([df,temp])

In [14]:
from sklearn.model_selection import StratifiedKFold
df['folds'] = 0
skf = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
for i, (_, test_index) in enumerate(skf.split(df[[' Destination Port']], df[' Label'])):
    df.iloc[test_index, -1] = i

/usr/local/lib/python3.7/dist-packages/sklearn/model_selection/_split.py:680: UserWarning:

The least populated class in y has only 1 members, which is less than n_splits=10.



In [15]:
df = pd.get_dummies(df)

In [16]:
col = [' Destination Port', ' Flow Duration', ' Total Fwd Packets',
       ' Total Backward Packets', 'Total Length of Fwd Packets',
       ' Total Length of Bwd Packets', ' Fwd Packet Length Max',
       ' Fwd Packet Length Min', ' Fwd Packet Length Mean',
       ' Fwd Packet Length Std', 'Bwd Packet Length Max',
       ' Bwd Packet Length Min', ' Bwd Packet Length Mean',
       ' Bwd Packet Length Std', 'Flow Bytes/s', ' Flow Packets/s',
       ' Flow IAT Mean', ' Flow IAT Std', ' Flow IAT Max', ' Flow IAT Min',
       'Fwd IAT Total', ' Fwd IAT Mean', ' Fwd IAT Std', ' Fwd IAT Max',
       ' Fwd IAT Min', 'Bwd IAT Total', ' Bwd IAT Mean', ' Bwd IAT Std',
       ' Bwd IAT Max', ' Bwd IAT Min', 'Fwd PSH Flags', ' Bwd PSH Flags',
       ' Fwd URG Flags', ' Bwd URG Flags', ' Fwd Header Length',
       ' Bwd Header Length', 'Fwd Packets/s', ' Bwd Packets/s',
       ' Min Packet Length', ' Max Packet Length', ' Packet Length Mean',
       ' Packet Length Std', ' Packet Length Variance', 'FIN Flag Count',
       ' SYN Flag Count', ' RST Flag Count', ' PSH Flag Count',
       ' ACK Flag Count', ' URG Flag Count', ' CWE Flag Count',
       ' ECE Flag Count', ' Down/Up Ratio', ' Average Packet Size',
       ' Avg Fwd Segment Size', ' Avg Bwd Segment Size',
       ' Fwd Header Length.1', 'Fwd Avg Bytes/Bulk', ' Fwd Avg Packets/Bulk',
       ' Fwd Avg Bulk Rate', ' Bwd Avg Bytes/Bulk', ' Bwd Avg Packets/Bulk',
       'Bwd Avg Bulk Rate', 'Subflow Fwd Packets', ' Subflow Fwd Bytes',
       ' Subflow Bwd Packets', ' Subflow Bwd Bytes', 'Init_Win_bytes_forward',
       ' Init_Win_bytes_backward', ' act_data_pkt_fwd',
       ' min_seg_size_forward', 'Active Mean', ' Active Std', ' Active Max',
       ' Active Min', 'Idle Mean', ' Idle Std', ' Idle Max', ' Idle Min']

In [17]:
train_df= df[df['folds'] != 5]
valid_df = df[df['folds'] == 5]

In [18]:
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()
train_df[col] = scaler.fit_transform(train_df[col])
valid_df[col] = scaler.transform(valid_df[col])

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:3678: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



**Model**

In [19]:
def create_ae_mlp(num_columns, num_labels, hidden_units, dropout_rates, ls = 1e-2, lr = 1e-3):
    
    inp = tf.keras.layers.Input(shape = (num_columns, ))
    x0 = tf.keras.layers.BatchNormalization()(inp)
    
    #Encoder
    encoder = tf.keras.layers.GaussianNoise(dropout_rates[0])(x0)
    encoder = tf.keras.layers.Dense(hidden_units[0])(encoder)
    encoder = tf.keras.layers.BatchNormalization()(encoder)
    encoder = tf.keras.layers.Activation('swish')(encoder)
    
    #Decoder
    decoder = tf.keras.layers.Dropout(dropout_rates[1])(encoder)
    decoder = tf.keras.layers.Dense(num_columns, name = 'decoder')(decoder)
    
    
    x_ae = tf.keras.layers.Dense(hidden_units[1])(decoder)
    x_ae = tf.keras.layers.BatchNormalization()(x_ae)
    x_ae = tf.keras.layers.Activation('swish')(x_ae)
    x_ae = tf.keras.layers.Dropout(dropout_rates[2])(x_ae)

    out_ae = tf.keras.layers.Dense(num_labels, activation = 'sigmoid', name = 'ae_action')(x_ae)
    
    #Multi Layer perceptron
    x = tf.keras.layers.Concatenate()([x0, encoder])
    x = tf.keras.layers.BatchNormalization()(x)
    x = tf.keras.layers.Dropout(dropout_rates[3])(x)
    
    for i in range(2, len(hidden_units)):
        x = tf.keras.layers.Dense(hidden_units[i])(x)
        x = tf.keras.layers.BatchNormalization()(x)
        x = tf.keras.layers.Activation('swish')(x)
        x = tf.keras.layers.Dropout(dropout_rates[i + 2])(x)
        
    out = tf.keras.layers.Dense(num_labels, activation = 'sigmoid', name = 'action')(x)
    
    model = tf.keras.models.Model(inputs = inp, outputs = [out_ae, out])
    model.compile(optimizer = tf.keras.optimizers.Adam(learning_rate = lr),
                  loss = {
                          'ae_action': tf.keras.losses.BinaryCrossentropy(label_smoothing = ls),
                          'action': tf.keras.losses.BinaryCrossentropy(label_smoothing = ls), 
                         },
                  metrics = { 
                             'ae_action': tf.keras.metrics.AUC(name = 'AUC'), 
                             'action': tf.keras.metrics.AUC(name = 'AUC'), 
                            }, 
                 )
    
    return model

In [20]:
train_df.iloc[:,-15:].columns

Index([' Timestamp_7/7/2017 4:06', ' Timestamp_7/7/2017 4:07',
       ' Timestamp_7/7/2017 4:08', ' Timestamp_7/7/2017 4:09',
       ' Timestamp_7/7/2017 4:10', ' Timestamp_7/7/2017 4:11',
       ' Timestamp_7/7/2017 4:12', ' Timestamp_7/7/2017 4:13',
       ' Timestamp_7/7/2017 4:14', ' Timestamp_7/7/2017 4:15',
       ' Timestamp_7/7/2017 4:16', 'External IP_205.174.165.205.174.165.70',
       'External IP_205.174.165.69', 'External IP_205.174.165.70',
       'External IP_205.174.165.71'],
      dtype='object')

In [21]:
params = {'num_columns': len(col), 
          'num_labels': 15, 
          'hidden_units': [96, 96, 896, 448, 448, 256], 
          'dropout_rates': [0.03527936123679956, 0.038424974585075086, 0.42409238408801436, 0.10431484318345882, 0.49230389137187497, 0.32024444956111164, 0.2716856145683449, 0.4379233941604448], 
          'ls': 0, 
          'lr':1e-3, 
         }

In [22]:
import tensorflow as tf
tf.random.set_seed(42)
import tensorflow.keras.backend as K
import tensorflow.keras.layers as layers
from tensorflow.keras.callbacks import Callback, ReduceLROnPlateau, ModelCheckpoint, EarlyStopping

In [23]:
batch_size = 64

In [24]:
fold = 5
ckp_path = f'JSModel_{fold}.hdf5'
model = create_ae_mlp(**params)
ckp = ModelCheckpoint(ckp_path, monitor = 'val_action_AUC', verbose = 0, 
                      save_best_only = True, save_weights_only = True, mode = 'max')
es = EarlyStopping(monitor = 'val_action_AUC', min_delta = 1e-4, patience = 20, mode = 'max', 
                   baseline = None, restore_best_weights = True, verbose = 0)
history = model.fit(train_df[col], [train_df.iloc[:,-15:],train_df.iloc[:,-15:]], 
                    validation_data = (valid_df[col], [ valid_df.iloc[:,-15:],valid_df.iloc[:,-15:]]), 
                    epochs = 20, batch_size = batch_size, callbacks = [ckp, es], verbose = True)
hist = pd.DataFrame(history.history)
score = hist['val_action_AUC'].max()

Epoch 1/20
589/589 [==============================] - 15s 14ms/step - loss: 0.6869 - ae_action_loss: 0.3566 - action_loss: 0.3303 - ae_action_AUC: 0.7220 - action_AUC: 0.7564 - val_loss: 0.6229 - val_ae_action_loss: 0.3027 - val_action_loss: 0.3202 - val_ae_action_AUC: 0.7839 - val_action_AUC: 0.7599
Epoch 2/20
589/589 [==============================] - 8s 13ms/step - loss: 0.5986 - ae_action_loss: 0.3042 - action_loss: 0.2944 - ae_action_AUC: 0.7880 - action_AUC: 0.8056 - val_loss: 0.5902 - val_ae_action_loss: 0.2905 - val_action_loss: 0.2997 - val_ae_action_AUC: 0.8153 - val_action_AUC: 0.7994
Epoch 3/20
589/589 [==============================] - 8s 13ms/step - loss: 0.5832 - ae_action_loss: 0.2955 - action_loss: 0.2876 - ae_action_AUC: 0.8046 - action_AUC: 0.8174 - val_loss: 0.5476 - val_ae_action_loss: 0.2789 - val_action_loss: 0.2688 - val_ae_action_AUC: 0.8337 - val_action_AUC: 0.8463
Epoch 4/20
589/589 [==============================] - 8s 13ms/step - loss: 0.5746 - ae_action_lo